# Minimal LangGraph Agent for SageMaker Studio

A simple LangGraph agent using AWS Bedrock Claude.

## Setup (Run from CLI before using this notebook)

```bash
./setup-inference-profile.sh sonnet
```

Copy the output ARN into `INFERENCE_PROFILE_ARN` in the next cell.

In [ ]:
#################################################
# CONFIGURATION
# Use the profile created by Bedrock IDE (from SageMaker Unified Studio)
#################################################

INFERENCE_PROFILE_ARN = "arn:aws:bedrock:us-west-2:159878781974:application-inference-profile/hsl5b7kh1279"
REGION = "us-west-2"

#################################################

In [ ]:
import importlib.metadata

packages = [
    "langchain",
    "langchain-core",
    "langgraph",
    "langchain-aws",
    "langchain-mcp-adapters",
    "mcp",
    "httpx",
    "boto3",
]

print("Pre-installed packages:")
print("-" * 50)
for pkg in packages:
    try:
        version = importlib.metadata.version(pkg)
        print(f"{pkg:30} {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{pkg:30} NOT INSTALLED")

## 2. Imports

In [ ]:
from typing import Literal
from datetime import datetime

from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

print("Imports successful!")

## 3. Define Tools

Simple tools for testing the agent's tool-calling capabilities.

In [ ]:
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b


tools = [get_current_time, add_numbers]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## 4. Initialize LLM

Uses the `INFERENCE_PROFILE_ARN` from the configuration cell above.

In [ ]:
# Uses INFERENCE_PROFILE_ARN and REGION from the configuration cell above

llm = ChatBedrockConverse(
    model=INFERENCE_PROFILE_ARN,
    provider="anthropic",  # Required when using ARN
    region_name=REGION,
    temperature=0,
)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized!")
print(f"Profile: {INFERENCE_PROFILE_ARN}")
print(f"Region: {REGION}")

## 5. Build the LangGraph Agent

A minimal ReAct-style agent with:
- `agent` node: calls the LLM
- `tools` node: executes tools
- Conditional edge: routes to tools or ends

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Determine whether to continue to tools or end."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    """Call the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# Build the graph
graph = StateGraph(MessagesState)

# Add nodes
graph.add_node("agent", call_model)
graph.add_node("tools", ToolNode(tools))

# Add edges
graph.add_edge(START, "agent")
graph.add_conditional_edges("agent", should_continue)
graph.add_edge("tools", "agent")

# Compile
agent = graph.compile()

print("Agent graph compiled successfully!")

## 6. Visualize the Graph (Optional)

In [ ]:
# Display the graph structure (requires graphviz)
try:
    from IPython.display import Image, display
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Graph visualization not available: {e}")
    print("\nGraph structure:")
    print("  START -> agent -> (tools -> agent) | END")

## 7. Run the Agent

In [ ]:
def run_agent(question: str):
    """Run the agent with a question and display the response."""
    print(f"Question: {question}")
    print("-" * 50)
    
    result = agent.invoke({
        "messages": [
            SystemMessage(content="You are a helpful assistant. Use tools when needed."),
            HumanMessage(content=question),
        ]
    })
    
    final_message = result["messages"][-1]
    print(f"\nResponse:\n{final_message.content}")
    return result

In [ ]:
# Test: Get current time
result = run_agent("What is the current time?")

In [ ]:
# Test: Math calculation
result = run_agent("What is 42 + 17?")

In [ ]:
# Test: Multiple tools
result = run_agent("What time is it and what is 100 + 200?")

## 8. Try Your Own Question

In [ ]:
# Enter your own question here
my_question = "Add 999 and 1, then tell me the time."

result = run_agent(my_question)